In [1]:
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import json
from json import JSONDecodeError, JSONEncoder
import pandas as pd
import pickle
import os
import collections
from sklearn import metrics
import sklearn
import numpy as np
# from model import spam_ham
import json
from flask import (
	Flask,
	request
)
from importlib import import_module, reload


In [22]:
class spam_ham:
    def __init__(self, data_path):
        self.data_path = data_path
    def data_read(self):
        f = open(self.data_path,'r')
        data = csv.reader(f)
        cat_map = {
                "ham": 0,
                "spam": 1
            }
        msgs, self.labels = [], []    
        for i,r in enumerate(data):
            if i == 0:
                continue
            label,msg = cat_map[r[0]],r[1].strip()
            msgs.append(msg)
            self.labels.append(label)
        tf_idf = TfidfVectorizer()
        tf_idf.fit(msgs)
        self.X = tf_idf.fit_transform(msgs)
        with open('../scripts/tf_idf.pkl','wb') as f:
            pickle.dump(tf_idf,f)   
        return self.X,self.labels,tf_idf,msgs
    def tf_idf_transform(self,text,tf_idf):
        features = tf_idf.transform(text)
        return self
    def train(self):
        clf = LogisticRegression()
        clf.fit(self.X,self.labels)
        with open('../scripts/clf.pkl','wb') as f:
            pickle.dump(clf,f) 
        return clf
    def Predict(model,X):
        return model.predict(X)[0]

In [23]:
path = '../data/SPAM_Data.csv'
sh = spam_ham(path)
X,label,tf_idf,msgs = sh.data_read()
# sh.tf_idf_transform(tf_idf)
mod = sh.train()
# sh.Predict(mod,X[0])

In [45]:
class spam_ham_predict:
    def __init__(self, clf_path,tf_idf_path):
        self.clf = pickle.load(open(clf_path,'rb'))
        self.tf_idf = pickle.load(open(tf_idf_path,'rb'))
    def Featurize(self,text):
        features = self.tf_idf.transform([text])
        return features
    def Predict(self,features):    
        return self.clf.predict(features)[0]

In [46]:
tf_idf_path = '../scripts/tf_idf.pkl'
clf_path = '../scripts/clf.pkl'
sh = spam_ham_predict(clf_path,tf_idf_path)
feats = sh.Featurize(msgs[0])
sh.Predict(feats)

0

In [74]:
# res = request.post('http://127.0.0.1:5000/predict', json={"text": "You are a winner U have been specially selected 2 receive ¬£1000 or a 4* holiday (flights inc) speak to a live operator 2 claim 0871277810910p/min (18+)"})
# request.po
json={"text": "You are a winner U have been specially selected 2 receive ¬£1000 or a 4* holiday (flights inc) speak to a live operator 2 claim 0871277810910p/min (18+)"}
X = tf_idf.transform(list(json.values()))
mod.predict(X.reshape(1,-1))

array([1])